<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Logo%20Scikit-learn.png?raw=true">
</p>


 # **<font color="DeepPink">Evaluación de modelos usando validación cruzada</font>**

<p align="justify">
👀 La validación cruzada en scikit-learn es una técnica utilizada para evaluar el rendimiento de un modelo de aprendizaje automático. Consiste en dividir el conjunto de datos en múltiples subconjuntos llamados "folds" (pliegues), entrenar el modelo en una combinación de estos pliegues y evaluar su rendimiento en los pliegues restantes. Esto se repite varias veces, de manera que cada pliegue sirva tanto como conjunto de entrenamiento como de conjunto de prueba en diferentes iteraciones del proceso.
<br><br>
La validación cruzada es útil porque permite utilizar todos los datos disponibles tanto para el entrenamiento como para la evaluación del modelo, lo que puede proporcionar una estimación más precisa del rendimiento del modelo en datos no vistos. También ayuda a mitigar el riesgo de sobreajuste, ya que el modelo se evalúa en múltiples conjuntos de datos de prueba diferentes.
<br><br>
En scikit-learn, la validación cruzada se puede realizar utilizando la clase <code>cross_val_score</code> del módulo <code>model_selection</code>. Esta clase permite especificar el modelo a evaluar, los datos de entrada y la estrategia de validación cruzada a utilizar (por ejemplo, validación cruzada k-fold, validación cruzada estratificada, etc.). Retorna una lista de puntuaciones de evaluación del modelo para cada iteración de la validación cruzada.
<br><br>
✅ Algunas de las estrategias de validación cruzada disponibles:
<br><br>
<ol align="justify">
<li>
<b>Validación cruzada k-fold</b>: Divide el conjunto de datos en k pliegues, entrena el modelo en k-1 pliegues y lo evalúa en el pliegue restante. Este proceso se repite k veces, utilizando cada pliegue como conjunto de prueba exactamente una vez.
</li>
<li>
<b>Validación cruzada estratificada</b>: Similar a la validación cruzada k-fold, pero garantiza que las proporciones de las clases en cada pliegue sean aproximadamente las mismas que en el conjunto de datos original.
</li>
</ol>


https://scikit-learn.org/stable/modules/cross_validation.html

<p align="justify">
<mark>Ejemplo teórico:</mark>
<br>
<br>

1. **Validación cruzada k-fold:**
   Supongamos que tenemos un conjunto de datos con 100 muestras y usamos validación cruzada k-fold con k=5. En cada iteración, 80 muestras se utilizan para entrenar el modelo y las 20 restantes se utilizan para probarlo. Este proceso se repite 5 veces, con diferentes divisiones de entrenamiento/prueba en cada iteración.

2. **Validación cruzada estratificada:**
   Es particularmente útil cuando se trabaja con conjuntos de datos desbalanceados, donde una clase es mucho más frecuente que otra. Esta técnica ayuda a garantizar que todas las clases estén representadas de manera equitativa en los conjuntos de entrenamiento y prueba.

   Consideremos un conjunto de datos donde el 80% de las muestras pertenecen a la clase A y el 20% a la clase B. Al aplicar validación cruzada estratificada, cada pliegue tendrá aproximadamente la misma proporción de muestras de clase A y clase B que el conjunto de datos original. Esto asegura una evaluación justa del modelo en todas las clases.

❤ https://scikit-learn.org/stable/

<p align="justify">
👀 En este Colab, seguiremos trabando con características numéricas y con el mismo conjunto de datos, pero ahora se discutiran los aspectos prácticos de evaluar el rendimiento de generalización de nuestro modelo, a través de la validación cruzada, en lugar de hacerlo solamente con una simple división del conjunto de datos, en un conjunto de datos de prueba y un conjunto de datos de entrenamiento.
</p>



In [ ]:
import numpy as np
import pandas as pd

In [ ]:
adult_census = pd.read_csv("https://raw.githubusercontent.com/cristiandarioortegayubro/BDS/main/datasets/adult_census.csv")

<p align="justify">
👀 Armando nuestro <code>DataFrame</code>...</p>

In [ ]:
adult_census.drop(columns=["education-num"], inplace=True)
adult_census.head()

,age,workclass,education,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25,Private,11th,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,HS-grad,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,Assoc-acdm,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,Some-college,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,Some-college,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


<p align="justify">
👀 Separamos la variable objetivo de las variables explicativas.
</p>


In [ ]:
y = adult_census["class"]
X = adult_census.drop(columns=["class"])

 # **<font color="DeepPink">Seleccionando las columnas numéricas</font>**

In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   workclass       48842 non-null  object
 2   education       48842 non-null  object
 3   marital-status  48842 non-null  object
 4   occupation      48842 non-null  object
 5   relationship    48842 non-null  object
 6   race            48842 non-null  object
 7   sex             48842 non-null  object
 8   capital-gain    48842 non-null  int64 
 9   capital-loss    48842 non-null  int64 
 10  hours-per-week  48842 non-null  int64 
 11  native-country  48842 non-null  object
dtypes: int64(4), object(8)
memory usage: 4.5+ MB


In [ ]:
numerical_columns = ["age", "capital-gain", "capital-loss", "hours-per-week"]
X_numeric = X[numerical_columns]

In [ ]:
X_numeric

,age,capital-gain,capital-loss,hours-per-week
0,25,0,0,40
1,38,0,0,50
2,28,0,0,40
3,44,7688,0,40
4,18,0,0,30
...,...,...,...,...
48837,27,0,0,38
48838,40,0,0,40
48839,58,0,0,40
48840,22,0,0,20


<p align="justify">
👀 Ahora podemos crear un modelo usando la herramienta <code>make_pipeline</code> para encadenar el preprocesamiento y el estimador en cada iteración de nuestra validación cruzada.
</p>


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

 # **<font color="DeepPink">La necesidad de validación cruzada</font>**

<p align="justify">
👀 En el Colab anterior, se dividieron los datos originales en un conjunto de datos de entrenamiento y un conjunto de datos de prueba. La puntuación de la generalización del modelo entonces, dependerá en general de la forma en que se hace esa división.
<br><br>
Una desventaja de hacer una sola división, es que no da información sobre esta variabilidad. Otro inconveniente, en un entorno donde la cantidad de datos es pequeña, es que los datos disponibles para el entrenamiento y los datos disponibles para la prueba serán aún más pequeña después de esa división.
<br><br>
✅ Entonces, podemos utilizar la validación cruzada.
<br><br>
La validación cruzada consiste en repetir el procedimiento de manera que el conjunto de datos de entrenamiento y el conjunto de datos de prueba sean diferentes en cada repetición. Las métricas de rendimiento de la generalización del modelo se recopilan para cada repetición. Como resultado, se puede evaluar la variabilidad del rendimiento del modelo.
<br><br>
Hay que tener en cuenta que existen varias estrategias de validación cruzada, y cada una de ellas define cómo repetir el procedimiento de ajuste <code>fit</code> y la evaluación <code>score</code>.
<br><br>
Por ahora, vamos a usar la estrategia <code>K-fold</code> que implica que todo el conjunto de datos se divide en $K$ particiones. El procedimiento de ajuste <code>fit</code> y la evaluación <code>score</code> se repite $K$ veces donde en cada iteración $K - 1$ las particiones se usan para ajustar el modelo.
<br><br>
👀 La siguiente figura ilustra esta estrategia <code>K-fold</code>.
</p>


<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/k-fold-001.png?raw=true" width="600">
</p>


<p align="justify">
👀 Esta figura muestra el caso particular de la estrategia de validación cruzada <code>K-fold</code>
<br><br>
Para cada división de validación cruzada, el procedimiento entrena un clon del modelo en todos los puntos rojos y evalua la puntuación del modelo en los azules. Como se mencionó anteriormente, hay una variedad de diferentes validaciones cruzadas. Por lo tanto, la validación cruzada es computacionalmente intensiva porque requiere entrenar varios modelos, en vez de entrenar solo uno.
<br><br>
En <code>scikit-learn</code>, la función <code>cross_validate</code> permite realizar la validación cruzada y necesita pasar el modelo, los datos y la variable objetivo. El parámetro <code>cv</code> define la estrategia de división, es decir, en cuanto se divide...
</p>


https://scikit-learn.org/stable/modules/cross_validation.html

In [ ]:
from sklearn.model_selection import cross_validate

In [ ]:
model = make_pipeline(StandardScaler(), LogisticRegression())

In [ ]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [ ]:
cv_result = cross_validate(model, X_numeric, y, cv=5)

In [ ]:
cv_result

{'fit_time': array([0.15263486, 0.14287543, 0.15722656, 0.13746428, 0.14785981]),
 'score_time': array([0.04040909, 0.03861451, 0.0436275 , 0.03715563, 0.03864408]),
 'test_score': array([0.79557785, 0.80049135, 0.79965192, 0.79873055, 0.80436118])}

<p align="justify">
👀 La salida de <code>cross_validate</code> es un diccionario de Python, que por defecto contiene tres entradas:
</p>


1. El tiempo para entrenar el modelo en los datos de entrenamiento para cada división, `fit_time`
1. El tiempo para predecir con el modelo en los datos de prueba para cada división, `score_time`
1. La puntuación predeterminada en los datos de prueba para cada división, `test_score`.


<p align="justify">
👀 Establecer en el parámetro <code>cv = 5</code> creó 5 divisiones distintas para obtener 5 variaciones para el conjunto de datos de entrenamiento y el conjunto de datos de prueba. Cada conjunto de datos de entrenamiento se utiliza para adaptarse a un modelo que luego se aplica al conjunto de datos de prueba correspondiente.
<br><br>
La estrategia por defecto cuando se configura <code>cv = int</code> es
la validación cruzada <code>K-fold</code> donde $K$ corresponde al número entero de divisiones. Establecer <code>cv = 5</code> o <code>cv = 10</code> es una práctica común, ya que es una buena compensación entre el tiempo de cálculo y la estabilidad de la variabilidad estimada.
<br><br>
Hay que tener en cuenta que, por defecto, la función <code>cross_validate</code> descarta los modelos $K$ que fueron entrenados en los diferentes subconjuntos superpuestos del conjunto de datos.
<br><br>
La meta de la validación cruzada no es entrenar un modelo, sino más bien estimar aproximadamente el rendimiento de un modelo que habría sido
entrenado al conjunto de entrenamiento completo, junto con una estimación de la variabilidad, que es la incertidumbre sobre la precisión.
<br><br>
👀 Ahora se pueden extraer las puntuaciones calculadas en la validación cruzada, con <code>cv_result</code> y calcular la precisión media y la
variación de la precisión.
</p>


In [ ]:
cv_result

{'fit_time': array([0.15263486, 0.14287543, 0.15722656, 0.13746428, 0.14785981]),
 'score_time': array([0.04040909, 0.03861451, 0.0436275 , 0.03715563, 0.03864408]),
 'test_score': array([0.79557785, 0.80049135, 0.79965192, 0.79873055, 0.80436118])}

In [ ]:
cv_result["test_score"].mean().round(4)

0.7998

In [ ]:
scores = cv_result["test_score"]

In [ ]:
type(scores)

numpy.ndarray

In [ ]:
scores

array([0.79557785, 0.80049135, 0.79965192, 0.79873055, 0.80436118])

In [ ]:
print("The mean cross-validation accuracy is: "f"{scores.mean():.4f} ± {scores.std():.3f}")

The mean cross-validation accuracy is: 0.7998 ± 0.003


<p align="justify">
👀 Hay que tener en cuenta que al calcular la desviación estándar de las puntuaciones de validación cruzada, podemos estimar la incertidumbre del rendimiento de nuestro modelo. Esta es la principal ventaja de la validación cruzada y puede ser crucial en la práctica, por ejemplo, al comparar diferentes modelos para determinar si uno es mejor que el otro.
<br><br>
En este caso particular, solo los primeros $2$ decimales parecen ser confiables. Si se observa en este Colab, se puede verificar que el rendimiento que obtenemos con la validación cruzada es compatible o similar al rendimiento que se obtiene con la sola división de los datos, en un conjunto de datos de prueba y un conjunto de datos de entrenamiento.

 # **<font color="DeepPink">Conclusiones</font>**

<p align="justify">
👀 En este colab nosotros:<br><br>
✅ Cargamos los datos de un archivo <code>CSV</code> usando <code>Pandas</code>.<br>
✅ Examinamos las variables numéricas.
<br>
✅ Hacemos la validación cruzada y comparamos.
</p>


<br>
<br>
<p align="center"><b>
💗
<font color="DeepPink">
Hemos llegado al final de nuestro colab, a seguir codeando...
</font>
</p>
